In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-02-20


Found rundate 2024-02-19 10:00:02 < 2024-02-20 at commit 092c6e37; returning commit 017cac4


'2024-02-20 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-02-20 10:00:03, NJSP was reporting 72 YTD deaths
Current YTD Deaths (2025-02-20 10:00:01-05:00): 66
Previous year YTD Deaths (adjusted; 2024-02-20 10:00:03): 72.19722712557474
Projected 2025 total: 677.465908176714
-8.6% change


FAUQStats cache miss: 2024, Tue Feb 20 10:00:03 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,East Oak St,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,27,State Highway 27,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 08:37:00-05:00
12953,08,Gloucester,0802,Deptford Twsp,47,State Highway 47,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-06 12:10:00-05:00
12928,15,Ocean,1505,Berkeley Twsp,9,State Highway 9 (Business),1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 18:44:00-05:00
12960,11,Mercer,1111,Trenton,29,State Highway 29,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-01-06 21:16:00-05:00
12930,13,Monmouth,1334,Neptune Twsp,33,State Highway 33,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2024-01-07 15:55:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,3,0,2,0,5
Bergen,3,0,1,0,4
Burlington,2,0,2,0,4
Camden,6,0,1,0,7
Cape May,2,0,0,0,2
Cumberland,2,0,1,0,3
Gloucester,2,0,0,0,2
Hudson,0,0,1,0,1
Mercer,1,0,2,0,3


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

13.8% through the year, 86.2% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,2,31,3
Bergen,driver,4,14,3
Burlington,driver,2,32,2
Camden,driver,2,19,6
Cumberland,driver,0,19,2
...,...,...,...,...
Somerset,cyclist,0,0,0
Sussex,cyclist,0,1,0
Cape May,crashes,0,9,2


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,51,2,34,1,18
Bergen,34,1,13,4,17
Burlington,46,1,32,2,11
Camden,40,0,28,3,11
Cape May,11,0,11,0,1
Cumberland,36,0,20,5,7
Essex,40,0,16,9,21
Gloucester,29,0,21,6,5
Hudson,16,1,8,1,7


In [9]:
projected.to_csv(PROJECTED_CSV)